# Calculate Interrater Reliability

## Setup

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import itertools
import os
import glob

In [2]:
# Set directories
irr_path = "/Users/tylund/Library/CloudStorage/Dropbox/1. Side Projects/2025.1-Deepfake Threat Landscape/3-tests/irr"

In [41]:
# Load data
irr_round = 'round2' # Define test round

files = glob.glob(os.path.join(irr_path, irr_round, '*.csv'))

df_1 = pd.read_csv(files[0])
df_2 = pd.read_csv(files[1])

## Data Preparation

In [4]:
import re

def expand_to_binary(df, code_cols, id_col='incident_id'):
    """
    Expands multiple code columns with delimited values into binary one-hot encoded columns.
    Handles multiple delimiters, lowercases, strips whitespace, and prefixes each binary column
    with the source column name for clarity.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with one or more code columns.
    code_cols : list of str
        Names of columns containing delimited code strings.
    id_col : str, default 'record_id'
        Column name identifying each record (preserved in the output).
    """
    df = df.copy()
    out_df = df[[id_col]].copy()

    # Normalize all column names
    df.columns = [col.strip().lower() for col in df.columns]

    # Normalize the record_col
    id_col = id_col.lower()

    # Make sure code_cols are lowercase to match normalized column names
    code_cols = [col.lower() for col in code_cols]
    
    all_binary_cols = []  # list to hold DataFrames for each code_col

    for code_col in code_cols:
        # Normalize delimiters and lowercase
        codes_series = (
            df[code_col]
            .fillna('')
            .astype(str)
            .apply(lambda x: re.sub(r'[;|/]', ',', x))
            .apply(lambda x: sorted(set([c.strip().lower() for c in x.split(',') if c.strip()])))
        )

        # Find all unique codes for this column
        unique_codes = sorted(set(code for codes in codes_series for code in codes))

        # Build a DataFrame for this code_col
        binary_data = pd.DataFrame(
            {f"{code_col}_{code}": codes_series.apply(lambda codes: int(code in codes))
             for code in unique_codes},
            index=df.index
        )

        all_binary_cols.append(binary_data)

    # Concatenate all binary columns at once with the record_id
    out_df = pd.concat([out_df] + all_binary_cols, axis=1)

    return out_df

In [5]:
def get_code_cols(df, start_col='Media Type'):
    """
    Retrieves a list of columns that include codes.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with one or more code columns.
    start_col : str
        Name of first code column.
    """
    df = df.copy()
    start_idx = df.columns.get_loc(start_col)
    code_cols = df.columns[start_idx:].tolist()

    return code_cols

In [6]:
def align_dfs(df1, df2, id_col='incident_id'):
    """
    Align two binary-coded data frames for IRR computation.

    - Ensures both data frames have the same record order (based on id_col)
    - Ensures both data frames have the same columns (union of all columns)
    - Fills missing columns with 0
    """
    df1 = df1.copy()
    df2 = df2.copy()

    # Normalize column names
    df1.columns = [col.lower() for col in df1.columns]
    df2.columns = [col.lower() for col in df2.columns]
    id_col = id_col.lower()

    # Filter for common IDs
    # Only compare records that exist in both datasets
    common_ids = set(df1[id_col]) & set(df2[id_col])
    
    if len(common_ids) == 0:
        raise ValueError("No common IDs found between the two datasets.")
        
    df1 = df1[df1[id_col].isin(common_ids)]
    df2 = df2[df2[id_col].isin(common_ids)]
    
    # Union of all binary columns (excluding id_col)
    all_cols = sorted(set(df1.columns) | set(df2.columns))
    all_cols = [c for c in all_cols if c.lower() != id_col]
    
    # Add missing columns filled with 0
    for col in all_cols:
        if col not in df1.columns:
            df1[col] = 0
        if col not in df2.columns:
            df2[col] = 0

    # Reorder columns: id_col first, then sorted binary columns
    df1 = df1[[id_col] + all_cols].sort_values(id_col).reset_index(drop=True)
    df2 = df2[[id_col] + all_cols].sort_values(id_col).reset_index(drop=True)

    # Print a warning if data was dropped
    if len(df1) < len(common_ids) or len(df2) < len(common_ids):
        print(f"Notice: Data restricted to {len(common_ids)} common records.")
    
    return df1, df2

In [42]:
# Create binary data frames
binary_df_1 = expand_to_binary(df_1,
                               code_cols = get_code_cols(df_1)
                              )

binary_df_2 = expand_to_binary(df_2,
                               code_cols = get_code_cols(df_2)
                              )

# Ensure dataframes are aligned for IRR tests
aligned_1, aligned_2 = align_dfs(binary_df_1, binary_df_2, id_col='incident_id')

## Calculate IRR

In [8]:
import krippendorff
from sklearn.metrics import cohen_kappa_score, jaccard_score

In [43]:
def compute_irr_metrics(df1, df2, id_col='incident_id'):
    """
    Compute IRR metrics (Cohen's Kappa, Krippendorff's Alpha, Jaccard similarity)
    between two aligned binary-coded DataFrames.

    Parameters
    ----------
    df1, df2 : pd.DataFrame
        Two aligned binary-coded DataFrames with the same rows and columns.
    id_col : str
        Column name for record identifier (will be excluded from calculations).

    Returns
    -------
    dict
        Dictionary with per-code and average values for all three IRR metrics.
    """
    df1 = df1.copy()
    df2 = df2.copy()

    # Drop the ID column
    X1 = df1.drop(columns=id_col)
    X2 = df2.drop(columns=id_col)

    code_cols = X1.columns.tolist()

    # Fill NaN with 0 for Kappa and Jaccard (treat missing as "not assigned")
    X1_filled = X1.fillna(0).astype(int)
    X2_filled = X2.fillna(0).astype(int)

    # Cohen's Kappa per code
    kappa_scores = {col: cohen_kappa_score(X1_filled[col], X2_filled[col]) for col in code_cols}
    average_kappa = np.mean(list(kappa_scores.values()))

    # Krippendorff's Alpha (use NaN for missing values)
    rater1 = X1.to_numpy().ravel()
    rater2 = X2.to_numpy().ravel()
    
    # Stack them vertically to create the matrix
    data_stack = np.vstack([rater1, rater2])
    
    kripp_alpha = krippendorff.alpha(reliability_data=data_stack, level_of_measurement='nominal')

    # Jaccard similarity per code
    jaccard_scores = {col: jaccard_score(X1_filled[col], X2_filled[col]) for col in code_cols}
    average_jaccard = np.mean(list(jaccard_scores.values()))

    return {
        'cohen_kappa_per_code': kappa_scores,
        'average_kappa': average_kappa,
        'krippendorff_alpha': kripp_alpha,
        'jaccard_per_code': jaccard_scores,
        'average_jaccard': average_jaccard
    }

In [44]:
irr_results = compute_irr_metrics(aligned_1, aligned_2, id_col='incident_id')

print("Average Cohen's Kappa:", irr_results['average_kappa'])
print("Krippendorff's Alpha:", irr_results['krippendorff_alpha'])
print("Average Jaccard:", irr_results['average_jaccard'])

Average Cohen's Kappa: 0.41248177112343204
Krippendorff's Alpha: 0.6359591500946309
Average Jaccard: 0.38553352925607925


In [45]:
from datetime import datetime

# Get current date as YYYYMMDD
today = datetime.today().strftime('%Y%m%d')

# Construct the filename dynamically
irr_fn = os.path.join(irr_path, f"irr_results_{today}.txt")

with open(irr_fn, "w") as f:
    f.write("Cohen's Kappa per code:\n")
    for code, score in irr_results['cohen_kappa_per_code'].items():
        f.write(f"{code}: {score:.3f}\n")
    
    f.write(f"\nAverage Cohen's Kappa: {irr_results['average_kappa']:.3f}\n")
    f.write(f"Krippendorff's Alpha: {irr_results['krippendorff_alpha']:.3f}\n\n")

## Manual Comparison

In [46]:
def get_disagreement_report(df1, df2, output_file, id_col='incident_id'):
    """
    Compares two already-aligned dataframes and creates sheets for a disagreement report.
    
    Input:
    - df1, df2: Dataframes containing the ID column and binary codes.
                (Must be the same shape/columns).
    - id_col: The name of the identifier column.
    """
    
    # 1. Detect Disagreements
    # Melt both to long format [ID, Variable, Value]
    melt1 = df1.melt(id_vars=id_col, var_name='variable', value_name='Rater_1')
    melt2 = df2.melt(id_vars=id_col, var_name='variable', value_name='Rater_2')
    
    # Merge on ID and Variable to compare side-by-side
    comparison = pd.merge(melt1, melt2, on=[id_col, 'variable'])
    
    # Filter for rows where values differ
    disagreements = comparison[comparison['Rater_1'] != comparison['Rater_2']].copy()
    
    if disagreements.empty:
        print("No disagreements found.")
        return

    # Sort specifically by ID for the detailed view
    disagreements = disagreements.sort_values(by=[id_col, 'variable'])

    # 2. Create Summaries
    # Summary A: By Record (Prioritize IDs with most issues)
    summary_by_id = disagreements.groupby(id_col).size().reset_index(name='disagreement_count')
    summary_by_id = summary_by_id.sort_values('disagreement_count', ascending=False)

    # Summary B: By Variable (Prioritize confusing definitions)
    summary_by_var = disagreements.groupby('variable').size().reset_index(name='disagreement_count')
    summary_by_var = summary_by_var.sort_values('disagreement_count', ascending=False)

    # 3. Export to Excel
    try:
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # Sheet 1: Summary by ID
            summary_by_id.to_excel(writer, sheet_name='Summary_by_ID', index=False)
            
            # Sheet 2: Summary by Variable
            summary_by_var.to_excel(writer, sheet_name='Summary_by_Variable', index=False)
            
            # Sheet 3: All Details
            disagreements.to_excel(writer, sheet_name='All_Details', index=False)
            
    except ImportError:
        print("Error: 'openpyxl' library is required to save Excel files.")

In [47]:
# Get current date as YYYYMMDD
today = datetime.today().strftime('%Y%m%d')

# Construct the filename dynamically
disagreement_fn = os.path.join(irr_path, f"disagreement_report_{today}.xlsx")

# Save disagreement report to csv
get_disagreement_report(aligned_1, aligned_2, disagreement_fn)